# Chatbot Tutorial

## 1. Preparations

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

## 2. Load & Preprocess Data

In [2]:
corpus_name = 'cornell_movie_dialogs_corpus'
corpus = os.path.join('data', corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as fr:
        lines = fr.readlines()
    for line in lines[:n]:
        print(line)

In [3]:
printLines(os.path.join(corpus, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


### Create formatted data file

Formatted data file: each line contains ***a tab-separated query sentence*** and ***a response sentence*** pair.

In [4]:
# Splits each line into a dictionary: 
# lines是字典，每个元素line：key=lineID, value=lineObj={lineID:xxx, characterID:xxx, movieID:xxx, character:xxx, text:xxx}
def loadLines(file, cols):
    lines = {}
    with open(file, 'r', encoding='iso-8859-1') as fr:
        for line in fr:
            values = line.split(' +++$+++ ')
            lineObj = {col: values[i] for i, col in enumerate(cols)}
            lines[lineObj['lineID']] = lineObj
    return lines

In [5]:
# Groups cols of lines from `loadLines` into conversations based on movie_conversations.txt
# conversations是列表，每个元素convObj: {col1:xxx, col2:xxx, ..., lines: [lineObj1, lineObj2, ..., lineObjm]}
def loadConversations(file, lines, cols):
    conversations = []
    with open(file, 'r', encoding='iso-8859-1') as fr:
        for line in fr:
            values = line.split(' +++$+++ ')
            convObj = {col: values[i] for i, col in enumerate(cols)}
            convObj['lines'] = [lines[lineId] for lineId in eval(convObj['utteranceIDs'])]
            conversations.append(convObj)
    return conversations

> 刘尧：训练数据是每个conversation中所有sentence生成的sentece对: <前一句话, 后一句话\>

In [6]:
# Extracts pairs of sentences from conversations
# qa_pair是列表，每个元素是sentence pair: [conv1_text1,conv1_text2], [1_2,1_3], [1_3,1_4], [1_4,1_5], ..., [2_1,2_2], [2_2,2_3], ...
def extractSentencePairs(conversations):
    qa_pair = []
    for conv in conversations:
        for i in range(len(conv['lines']) - 1):  # Ignore the last line (no answer for it)
            inputLine = conv['lines'][i]['text'].strip()
            targetLine = conv['lines'][i + 1]['text'].strip()
            if inputLine and targetLine:
                qa_pair.append([inputLine, targetLine])
    return qa_pair

In [7]:
datafile = os.path.join(corpus, 'formatted_movie_lines.txt')
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))  # 有时间好好研究这句话！！！

lines = {}
conversations = []
MOVIE_LINES_COLS = ['lineID', 'characterID', 'movieID', 'character', 'text']
MOVIE_CONVERSATIONS_COLS = ['character1ID', 'character2ID', 'movieID', 'utteranceIDs']

In [8]:
print('\nProcessing corpus and loading conversations...')
lines = loadLines(os.path.join(corpus, 'movie_lines.txt'), MOVIE_LINES_COLS)
conversations = loadConversations(os.path.join(corpus, 'movie_conversations.txt'), lines, MOVIE_CONVERSATIONS_COLS)


Processing corpus and loading conversations...


In [9]:
print('\nWriting newly formatted file ...')
with open(datafile, 'w', encoding='utf8') as fw:
    writer = csv.writer(fw, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)
print('\nSample lines from file: ')
printLines(datafile)


Writing newly formatted file ...

Sample lines from file: 
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a 

### Load and trim data

> 刘尧：把vocabulary及其附属或衍生变量以及相关method封装抽象成Class！这样既保护数据又方便使用！好好好！！！

Note that we are dealing with sequences of **words**, we should create a **vocabulary**: mapping each unique word that we encounter in our dataset to an index value.

For this we define a ***Vocabulary*** class, which has 5 attributes and 3 methods:

- 5 attributes

    - **word2index**: A mapping from each word to index

    - **index2word**: A reverse mapping from index to each word

    - **word2count**: A mapping from each word to its count

    - num_words: A total word count
    
    - trimmed: If infrequently seen words are trimmed

- 3 methods

    - **addWord**: Adding a word to the vacabulary

    - addSentence: Adding all words in a sentence

    - trim: Trimming infrequently seen words

In [10]:
# Default word tokens
PAD_TOKEN = 0
SOS_TOKEN = 1  # Start of sentence
EOS_TOKEN = 2  # End of sentence 

MAX_LENGTH = 10

In [11]:
class Vocabulary(object):
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}  # 不默认包含PAD,SOS,EOS这仨
        self.word2count = {}
        self.index2word = {PAD_TOKEN: 'PAD', SOS_TOKEN: 'SOS', EOS_TOKEN: 'EOS'}
        self.num_words = 3  # SOS, EOS, PAD
        
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words    # 添加的word，其index依次往后排
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        
        keep_words = [k for k, v in self.word2count.items() if v >= min_count]
        print(f'keep_words {len(keep_words)} / {len(self.word2index)} = {len(keep_words) / len(self.word2index): .4f}')
        
        # Reinitializa dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_TOKEN: 'PAD', SOS_TOKEN: 'SOS', EOS_TOKEN: 'EOS'}
        self.num_words = 3
        
        for word in keep_words:
            self.addWord(word)
            
        self.trimmed = True

Some data preprocessing:

- **unicodeToAscii**: Convert the Unicode strings to ASCII

- **normalizeString**: Convert all letters to lowercase and trim all non-letter characters except for basic punctuation

- **filterPairs**: Filter sentences with length greater than the *MAX_LENGTH* threshold

> 刘尧：这些常规的预处理，最好封装成一个个function，以方便使用！可以放在**Coding通用工具脚本**里！

In [12]:
def unicodeToAscii(s):
    """Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427"""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [13]:
def normalizeString(s):
    """Lowercase, trim, and remove non-letter characters"""
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'([.!?])', r' \1', s)      # 把.!?三个标点符号替换为？
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)  # 把字母和.!?之外的character替换为空格
    s = re.sub(r'\s+', r' ', s).strip()    # 把替换为空格
    return s

In [14]:
def readVocs(datafile, corpus_name):
    """Read <query, response> pairs and return a Vocabulary object"""
    lines = open(datafile, encoding='utf8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in line.split('\t')] for line in lines]
    voc = Vocabulary(corpus_name)
    return voc, pairs

In [15]:
def filterPair(pair):
    """Return True iff both sentences in pair are under the MAX_LENGTH threshold"""
    return len(pair[0].split(' ')) < MAX_LENGTH and len(pair[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    """Filter pairs using filterPair function"""
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def loadPrepareData(corpus, corpus_name, datafile, save_dir):  # corpus, save_dir 在哪里使用的！？！
    """Using the functions above, return a populated Vocabulary object and pairs list"""
    print('Start preparing training data ...')
    vocabulary, pairs = readVocs(datafile, corpus_name)
    print('Read {!s} sentence pairs'.format(len(pairs)))
    pairs = filterPairs(pairs)
    print('Trimmed to {!s} sentence pairs'.format(len(pairs)))
    print('Counting words ...')
    for pair in pairs:
        vocabulary.addSentence(pair[0])
        vocabulary.addSentence(pair[1])
    print('Counted words: ', vocabulary.num_words)
    return vocabulary, pairs

In [17]:
# Load/Assemble vocabulary and pairs
save_dir = os.path.join('data', 'save')
vocabulary, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

Start preparing training data ...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words ...
Counted words:  18008


In [18]:
for pair in pairs[:10]:
    print(pair)

['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


Another tactic that is beneficial to achieving faster convergence during training is **trimming rarely used words out of our vocabulary**. 

Decreasing the feature space will also soften the difficulty of the function that the model must learn to approximate.

We will do this as a two-stage process:
    
- Trim words used under *MIN_COUNT* threshold using the *Vocabulary.trim* function

- Filter out pairs with trimmed words

> 刘尧：事先从Vocabulary中定义并删除不常见的word，即**OOV的word**，随后从训练数据中删除这些OOV的word！ 

> 刘尧：疑问：模型应用时遇到OOV的word咋办？？？跟训练一样，应用前也先使用trimRareWords来处理一下！？

In [19]:
MIN_COUNT = 3
def trimRareWords(vocabulary, pairs, MIN_COUNT):
    """基于MIN_COUNT，删除vocabulary中不常见的word，并从训练/应用数据中删除带有不常见word的pairs"""
    vocabulary.trim(MIN_COUNT)
    
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input, keep_output = True, True
        
        # 判断pairs中2个句子中是否存在OOV的word，一旦存在，则删除当前pairs
        for word in input_sentence.split(' '):
            if word not in vocabulary.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in vocabulary.word2index:
                keep_output = False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
            
    print(f'Trimmed from {len(pairs)} pairs to {len(keep_pairs)}, {len(keep_pairs) / len(pairs): .4f} of total')
    return keep_pairs

In [20]:
pairs = trimRareWords(vocabulary, pairs, MIN_COUNT)

keep_words 7823 / 18005 =  0.4345
Trimmed from 64271 pairs to 53165,  0.8272 of total


## 3. Prepare Data for Models

Already done: preparing and massaging data into a nice vocabulary object and list of sentence pairs.

Not done yet: preparing numerical torch tensors as model inputs.

When batch_size=1, all we have to do is convert words in sentence pairs to their corresponding indexes from vocabulary and feed this to the models

When batch_size>1, which can **speed up training and/or leverage GPU parallelization capabilities**, we must be mindful of the variation of sentence length in our batches. 

To accommodate sentences of different sizes in the same batch, we will make our batched input tensor of shape *(max_length, batch_size)*.

If we simply convert sentences to tensors by converting words to their indexes (*indexesFromSentence*) and zero-pad, our tensor would have shape (*batch_size, max_length*). However, we need to be able to **index our batch along time(为啥子？)**, and **across all sequences in the batch**. 

Therefore, we transpose our input batch shape to (*max_length, batch_size*), so that indexing across the 1st dimension returns **a time step across all sentences** in the batch (*zeroPadding*).

> Shape Transpose: (*batch_size, max_length*) --> (*max_length, batch_size*)

![001](./image/seq2seq_batches.png)

> 刘尧：这么一大段到底要讲个啥子玩意儿？！为什么一定要Transpose一下？！

In [21]:
def sentenceToIndexes(vocabulary, sentence):
    """sentence向量化"""
    return [vocabulary.word2index[word] for word in sentence.split(' ')] + [EOS_TOKEN]

> 刘尧：疑惑：zeroPadding函数中，取seq_batch中的max_length_batch进行padding，那么不同batch的max_length_batch应该无法保证一样吧？

> 刘尧：疑惑：或者本来就不必一样了，因为经Transpose之后，max_length_batch为第1维，**batch_size变成第2维，所有batch的batch_size必然一样**，所以可直接输入到network中！？

> 刘尧：疑惑：以上是不是就是进行Transpose的重要原因！？！貌似与Keras或其他案例中不一样哎？

In [22]:
def zeroPadding(seq_batch, fillvalue=PAD_TOKEN):
    """按seq_batch中最长的seq的长度进行zero-padding，随后进行Transpose"""
    return list(itertools.zip_longest(*seq_batch, fillvalue=fillvalue))  # 内部隐含了Transpose操作！！

In [23]:
def seqsToMask2D(seqs, value=PAD_TOKEN):
    """判断seqs中每个seq的每个token是否取值为value(是=0 否=1)，把seqs转化为0和1的二维mask"""
    return [[int(token != value) for token in seq] for seq in seqs]

以下三个函数 *tempVar, inputVar, outputVar* 用于转化 sentence 数据：最普通的 sentences (list of sentence) --> Tensor of shape (batch_size, max_length_batch)

In [24]:
def sentencesToTensor(sentences, vocabulary):
    """对序列中每个sentence进行向量化和0-padding操作，随后整个序列转化为Tensor"""
    indexes_batch = [sentenceToIndexes(vocabulary, sentence) for sentence in sentences]  # 同一batch中的sentence向量化
    padList = zeroPadding(indexes_batch)    # zero-pad同一batch中的向量
    padVar = torch.LongTensor(padList)      # torch.LongTensor: dtype=torch.int64/torch.long所对应的CPU Tensor
    return indexes_batch, padList, padVar

def inputToTensor(sentences, vocabulary):
    """Returns padded input sequence tensor and lengths"""
    indexes_batch, padList, padVar = sentencesToTensor(sentences, vocabulary)
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    return padVar, lengths

def targetToTensor(sentences, vocabulary):
    """Returns padded target sequence tensor, padding mask, and max target length"""
    indexes_batch, padList, padVar = sentencesToTensor(sentences, vocabulary)
    max_length = max([len(indexes) for indexes in indexes_batch])
    mask2d = torch.ByteTensor(seqsToMask2D(padList))
    return padVar, max_length, mask2d

以下函数 *pairBatch2trainData* 用于转化 sentence pair 数据：pair_batch --> input sentences 和 target sentences --> 相应的 Tensor

In [25]:
def pairBatch2trainData(vocabulary, pair_batch):
    """Returns all items for a given batch of pairs"""
    pair_batch.sort(key=lambda x: len(x[0].split(' ')), reverse=True)  # 对pair_batch，按每个pair中第1个sentece的word数目倒序排序各pair
    input_batch, target_batch = zip(*pair_batch)                       # 解压/拆解每个pair
    inp, lengths = inputToTensor(input_batch, vocabulary)
    target, max_length, mask = targetToTensor(target_batch, vocabulary)
    return inp, lengths, target, max_length, mask

> 刘尧：感悟：数据前期操作处理如向量化、padding等可在转化为Tensor之前开展，以方便操作，待所有前期操作结束后，在输入模型进行训练前，再转化为Tensor！

> 刘尧：疑惑：这些前期操作，是不是在Tensor中有相应的便捷API？那么，需要先转化为Tensor？

In [26]:
# Example for validation
inp, lengths, target, max_length, mask = pairBatch2trainData(vocabulary, [random.choice(pairs) for _ in range(5)])
print("input:", inp)
print("input_lengths:", lengths)
print("target:", target)
print("max_target_len:", max_length)
print("mask:", mask)

input: tensor([[1329,   50, 3309,  787,  100],
        [  25,   12,  758,    4,    6],
        [ 310,  109,    4,    2,    2],
        [  76,    4,    2,    0,    0],
        [ 177,    2,    0,    0,    0],
        [  53,    0,    0,    0,    0],
        [2009,    0,    0,    0,    0],
        [1030,    0,    0,    0,    0],
        [  70,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
input_lengths: tensor([10,  5,  4,  3,  3])
target: tensor([[ 116,   25, 2818, 5093, 2007],
        [ 117,  200,    6,    2,   25],
        [ 341,  677,    2,    0,    8],
        [   4,   12,    0,    0,   40],
        [   2, 4849,    0,    0,  282],
        [   0,  227,    0,    0,    4],
        [   0,    4,    0,    0,    4],
        [   0,    2,    0,    0,    4],
        [   0,    0,    0,    0,    2]])
max_target_len: 9
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 0, 0, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
     

## 4. Define Models

### Seq2Seq Model

The goal of a seq2seq model is to take **a variable-length sequence** as an input, and return **a variable-length sequence** as an output using a fixed-sized model. By using 2 separate RNN together, we can accomplish this task. 

The fist RNN is an **encoder**, which encodes a variable-length input sequence to a fixed-length context vector. In theory, this context vector (the final hidden layer of the RNN) will contain semantic information about the input sequence. 

> **encoder**: \[X1,X2,...,Xn\](A variable-length sequence) --> A fixed-length context vector

The second RNN is a **decoder**, which takes an input word and the context vector, and returns a guess for the next word in the sequence and a hidden state to use in the next iteration.

> **decoder**: Xn(An input word) + A fixed-length context vector --> Y1(A guess for the next word) + A hidden state

![001](./image/seq2seq_ts.png)

### Encoder

The encoder RNN iterates through the input sequence one token ***Xi*** (e.g. word) at a time, at each time step ***i*** outputting an "output" vector ***Yi*** and a "hidden state" vector ***Si***. The hidden state vector ***Si*** is then passed to the next time step ***i+1***, while the output vector ***Yi*** is recorded.

> At each time step ***i***: ***Xi*** --> ***Yi*** (recorded) + ***Si*** (passed to step ***i+1***)

The encoder transforms the context it saw at each point in the sequence into **a set of points in a high-dimensional space**, which the decoder will use to generate a meaningful output for the given task.

The heart of encoder is a multi-layered GRU. We will use a bidirectional variant of the GRU, meaning that there are essentially 2 independent RNNs: one is fed the input sequence in normal sequential order, and the other is fed in reverse order.

The outputs of each network are summed at each time step ***i***. Using a bidirectional GRU will give us the advantage of encoding both past and future context.

Bidirectional RNN:

![](./image/RNN-bidirectional.png)

Note that:

- An ***embedding layer*** is used to encode our word indices in an arbitrarily sized feature space. For our models, this layer will map each word to a feature space of size *hidden_size*.

- If passing a padded batch of sequences to an RNN module, we must **pack and unpack padding** around the RNN pass using *torch.nn.utils.rnn.pack_padded_sequence* and *torch.nn.utils.rnn.pad_packed_sequence* respectively.

**Computation Graph:**

- Convert word indexes to embeddings

- Pack padded batch of sequences for RNN module

- Forward pass through GRU

- Unpack padding

- Sum bidirectional GRU outputs

- Return output and final hidden state

**Inputs:**

- *input_seq*: Batch of input sentences; shape=***(max_length, batch_size)***

- *input_lengths*: List of sentence lengths corresponding to each sentence in the batch; shape=***(batch_size)***

- *hidden*: Hidden state; shape=***(n_layers x num_directions, batch_size, hidden_size)***

**Outputs:**

- *outputs*: Output features from the last hidden layer of the GRU (sum of bidirectional outputs); shape=***(max_length, batch_size, hidden_size)***

- *hidden*: Updated hidden state from GRU; shape=***(n_layers x num_directions, batch_size, hidden_size)*** (same shape with *hidden* above in Inputs)

**几个注意点：**

- embedding 位于 EncoderRNN 之外额外处理

- nn.GRUCell 和 nn.GRU 有区别，前者是多个 GRUCell 组合在一起，后者才是 Encoder 中展示的那样，多个 GRU 串联在一起，可设置单向或双向，是 Encoder 的核心和主体

- pack_padded_sequence 和 pad_packed_sequence 这2个函数有意思，要好好研究一下到底在干啥子

- 输入中的 hidden 是所谓的 ***a0***，当n_layers=1时，Output 是 ***a1,a2,...,atx,...,aTx***，输出中的 hidden 是 ***aTx***

In [27]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        # Initialize GRU  注意nn.GRU与nn.GRUCell的区别！
        # input_size和hidden_size都设置为hidden_size，因为input size is a word embedding with number of features==hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)                                           # Convert word indexes to embeddings
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)      # Pack padded batch of sequences for RNN module
        outputs, hidden = self.gru(packed, hidden)                                     # Forward pass through GRU
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)                   # Unpack padding
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]  # Sum bidirectional GRU outpus   此时应该是对hidden进行sum吧或拼接吧？
        return outputs, hidden

**Encoder** (来自Coursera)

已知：a<0\>,  a<Tx+1\>, x<1\>, x<2\>, ..., x<tx\>, ...x<Tx\> 即输入sequence
    
流程：x<tx\> + a<tx-1\> --(BiLSTM/BiGRU)--> a<tx\>, y^<tx\>
    
结果：a<tx\> = (a<tx\>->, a<tx\><-) 即原始单词x<tx\>激活值，用于计算Attention和c<ty\>

### Decoder

The decoder RNN generates the response sentence in a **token-by-token** fashion. It uses the **encoder's context vectors**, and internal hidden states to generate the next word in the sequence. It continues generating words until it outputs and EOS_TOKEN.

> Token-by-token: context vectors + hidden states --> next word until EOS_TOKEN

### Attention Layer

A common problem with a vanilla seq2seq decoder is that if we **rely solely on the context vector** to encode the entire input sequence's meaning, it is likely that we will have information loss. This is especially the case when dealing with long input sequences, greatly limiting the capability of our decoder.

Attention mechanism can combat this, which allows the decoder to **pay attention to certain parts of the input sequence**, rather than using the entire fixed context at every step.

At a high level, attention is calculated using the decoder's current hidden state and the encoder's outputs (图片中是decoder's inputs ?). The output attention weights have the same shape as the input sequence, allowing us to multiply them by the encoder outputs, giving us **a weighted sum which indicates the parts of encoder output to pay attention to**.

> decoder's current hidden state + decoder's inputs --> attention weights

> attention weights x encoder outputs --> A weighted sum (attended encoder outputs)

![](./image/attn2.png)

An improvement is creating "Global attention". The key difference is that with "Global attention", we consider **all of the encoder's hidden states**, as opposed to "Local attention", which only considers **the encoder's hidden state from the current time step**. Another difference is that with "Global attention", we calculate attention weights, or energies, using the hidden state of the decoder from the current time step only. ...

Overall, the "Global attention mechanism" can be summarized by the following figure. Note that we will implement the "Attention Layer" as a separate *nn.Module* called *Attn*. The output of this module is a softmax normalized weights tensor of shape *(batch_size, 1, max_length)*.

![](./image/global_attn.png)

In [28]:
class Attn(nn.Module):
    """Attention Layer，Encoder所有Step的状态，对Decoder中某步ty状态的权重alpha"""
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        if method not in ['dot', 'general', 'concat']:
            raise ValueError(method, 'is not an appropriate attention method.')
        self.method = method
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn_nn = nn.Linear(self.hidden_size, hidden_size)      # 小神经网络中的Dense
        elif self.method == 'concat':
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            self.attn_nn = nn.Linear(self.hidden_size * 2, hidden_size)
    
    def forward(self, hidden, encoder_outputs):
        """
        ARG
            hidden=decoder中的s<ty-1>，是单步ty的状态
            encoder_outputs=encoder中的a(a<1>,a<2>,...,a<tx>,...,a<Tx>)，是所有Step的状态
            score=score(hidden, encoder_outputs)
        RETURN
            alpha<ty,:>，表示Encoder所有Step的状态对单步ty的权重
        """
        if self.method == 'dot':
            score = hidden * encoder_outputs
        elif self.method == 'general':
            score = hidden * self.attn_nn(encoder_outputs)
        elif self.method == 'concat':
            cated = torch.cat((hidden.expand(encoder_outputs.size(0), -1, -1), encoder_outputs), 2)
            score = self.v * self.attn_nn(cated).tanh()        # 
        attn_energies = torch.sum(score, dim=2).t()            # Transpose max_length and batch_size dimensions                    
        alpha = F.softmax(attn_energies, dim=1).unsqueeze(1)   # The softmax normalized probability scores   这个softmax是小神经网络最后的Softmax
        return alpha

**Attention** (来自coursera，只是concat模式) 与coursera中学的有一些差别！

已知：s<0\>

流程：s<ty-1\> + a<tx\> --(Dense)--> e<ty,tx\>  --(Softmax)--> alpha<ty,tx\> (即Attention)
    
流程：alpha<ty,tx\> + a<tx\> --(加权之和)-->  c<ty\>

结果：c<ty\> 即Context，用于Decoder中输入给LSTM

注意：a<tx\>两次使用：与s<ty-1\>拼接，与alpha<ty,tx\>加权求和

注意：c<ty\>是承上启下，承Encoder的结果，启Decoder的输入

### AttentionDecoder

For the decoder, we will manually feed our batch **one time step at a time**. This means that our embedded word tensor and GRU output will both have shape ***(1, batch_size, hidden_size)***.

**Computational Graph:**

- Get embedding of current input word

- Forward through unidirectional GRU

- Calculate attention weights from the current GRU output from the step above

- Multiply attention weights to encoder outputs to get new "weighted sum" context vector

- Concatenate weighted context vector and GRU output using Luong eq. 5

- Predict next word using Luong eq. 6 (without softmax)

- Return output and final hidden state

**Inputs:**

- *input_step*: **one time step (one word)** of input sequence batch; shape=***(1, batch_size)***

- *last_hidden*: final hidden layer of GRU; shape=*(n_layers x num_directions, batch_size, hidden_size)*

- *encoder_outputs*: encoder model's output; shape=*(max_length, batch_size, hidden_size)*

**Outputs:**

- *output*: softmax normalized tensor giving probabilities of each word being the correct next word in the decoded sequence; shape=*(batch_size, vocabulary.num_words)*

- *hidden*: final hidden state of GRU; shape=*(n_layers x num_directions, batch_size, hidden_size)*

In [29]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_method, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
        
        # Keep for reference
        self.attn_method = attn_method
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)          # 干啥子用的？
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))  # Decoder中的一个RNN Cell???
        self.attn = Attn(attn_method, hidden_size)            # Attention Layer 使用在此！
        self.linear1 = nn.Linear(hidden_size * 2, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    # Note: we run this one step (word) at a time
    def forward(self, input_step, last_hidden, encoder_outputs):
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        
        rnn_output, hidden = self.gru(embedded, last_hidden)         # Forward through GRU
        attn_weights = self.attn(rnn_output, encoder_outputs)        # Calculate attention weights from the current GRU output
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  # Multiply attention weights to encoder outputs
        
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        concat_input = torch.cat((rnn_output.squeeze(0), context.squeeze(1)), 1)  # 为什么之后没有再输入到下一个GRU中？？？
        concat_output = torch.tanh(self.linear1(concat_input))
        
        # Predict next word using Luong eq. 6
        output = self.linear2(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden      # hidden随后在下一Step时作为last_hidden继续输入至LuongAttnDecoderRNN中

## 5. Define Training Procedure

### Masked loss

Since we are dealing with batches of padded sequences, we can't simply consider all elements of the tensor when calculating loss. We define *maskNLLLoss* to calculate loss based on decoder's output tensor, the target tensor, and a binary mask tensor **describing the padding of the target tensor**. This function calculates the average negative log likelihood of **the elements that corresponding to a 1 in the mask tensor**.

> 刘尧：嘛意思？为什么要有mask！？

In [30]:
def maskNLLLoss(inp, target, mask):
    crossEntropy = - torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, mask.sum().item()

### Single training iteration

Single training iteration means **a single batch** of inputs.

We will use 2 clever tricks to aid in convergence:

- **Teacher forcing**: At some probability, set by *teacher_forcing_ratio*, we use the **current target word** as the decoder's next input rather than using the **decoder's current guess**. This technique aids in more efficient training. However, it can lead to model **instability during inference**, as the decoder may not have a sufficient chance to truly craft its own output sequences during training.

- **Gradient clipping**: This is a commonly used technique for countering the "exploding gradient" problem. In essence, by clipping or thresholding gradients to a maximum value, we prevent the gradients from **growing exponentially and either overflow (NaN), or overshoot steep cliffs in the cost function**.

**Sequence of Operations:**

1. Forward pass entire input batch through encoder.

2. Initialize decoder inputs as SOS_TOKEN, and hidden state as the encoder's final hidden state.

3. Forward input batch sequence through decoder one time step at a time.

4. If teacher forcing: set the current target as next decoder input; else: set current decoder output as next decoder input.

5. Calculate and accumulate loss.

6. Perform backpropagation.

7. Clip gradients.

8. Update encoder and decoder model parameters.

> 刘尧：虽然实际上，对于RNN Modules (RNN, LSTM, GRU)，我们 **one time-step at a time** 地迭代着处理，每次都计算当前的 hidden state，然后开始下一次迭代，但我们仍可像对待 non-recurrent layers 那样，**一次性输入 entire input sequence (or batch of sequences)**。当然，我们也可手动进行每次的迭代，就像在 decoder 中做的那样。

In [31]:
def train(inp, lengths, target, mask, max_target_len, encoder, decoder, embedding, 
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Set device options
    inp = inp.to(device)
    lengths = lengths.to(device)
    target = target.to(device)
    mask = mask.to(device)
    
    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0
    
    # 1. Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(inp, lengths)
    
    # 2.1 Initialize decoder input (start with SOS_TOKEN for each sentence)
    decoder_input = torch.LongTensor([[SOS_TOKEN for _ in range(batch_size)]])  # batch中每个sequence都是SOS_TOKEN
    decoder_input = decoder_input.to(device)
    
    # 2.2 Initialize decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[: decoder.n_layers]  # n_layers ???
    
    # 4.1 Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    # 3. Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):  # max_target_len表示共有多少个time step
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # 4.2 Teacher forcing: next input is current target
            decoder_input = target[t].view(1, -1)
            
            # 5. Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # 4.2 No teacher forcing: next input is decoder's own current output, that is, its guess ?
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)  # 为什么上面没有？
            
            # 5. Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
            
    # 6. Perform backpropagation
    loss.backward()
    
    # 7. Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    # 8. Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses) / n_totals

### Training iteration

It's finally time to tie the full training procedure together with the data. 

One thing to note is that when we save model, we save a tarball containing **the encoder and decoder state_dicts(parameters), the optimizer's state_dicts, the loss, the iteration**, etc. 

Saving model in this way will give us the ultimate flexibility with the checkpoint. After loading a checkpoint, we will be able to use the model parameters to **run inference, or continue training right where we left off**.

In [32]:
def trainIters(model_name, vocabulary, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, 
               decoder_n_layers, save_dir, n_iterations, batch_size, print_every, save_every, clip, corpus, checkpoint_file):
    
    # Load batches for each iteration
    training_batches = [pairBatch2trainData(vocabulary, [random.choice(pairs) for _ in range(batch_size)]) for _ in range(n_iterations)]
    
    # Initializations
    print('Initializing ...')
    next_iteration = 0
    print_loss = 0
    if checkpoint_file:
        # 难道checkpoint只在此处使用？只用到iteration这一个信息？？？不需要获得model和optimizer的state_dict然后基于此继续训练吗？？
        checkpoint = torch.load(checkpoint_file)
        next_iteration = checkpoint['iteration'] + 1
        # 此处不需要从checkpoint中获取encoder,decoder,encoder_optimizer,decoder_optimizer,embedding等吗！？！比如encoder.load_state_dict(checkpoint['encoder'])
        
    # Training loop
    print('Training ...')
    for i in range(next_iteration, n_iterations):
        training_batch = training_batches[i]
        inp, lengths, target, max_target_len, mask = training_batch  # Extract fields from batch
        
        # Run a training interation with batch
        loss = train(inp, lengths, target, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss
        
        # Print progress
        if (i + 1) % print_every == 0:
            print_loss_avg = print_loss / print_every
            print(f'Iteration: {i + 1}; Percent complete: {(i + 1) / n_iterations: .1f}%; Average loss: {print_loss_avg: .4f}')
            print_loss = 0
        
        # Save checkpoint
        if (i + 1) % save_every == 0:
            directory = os.path.join(save_dir, model_name, corpus_name, f'{encoder_n_layers}-{decoder_n_layers}_{hidden_size}')
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': i,
                'encoder': encoder.state_dict(),
                'decoder': decoder.state_dict(),
                'encoder_optimizer': encoder_optimizer.state_dict(),
                'decoder_optimizer': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': vocabulary.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, f'{i}_checkpoint.tar'))

## 6. Define Inference

We should define how we want the model to decode the encoded input.

### Greedy decoding

Greedy decoding is the decoding method that we use during training when we are **NOT** using teacher forcing. In other words, for each time step, we simply choose **the word from decoder_output with the highes softmax value**. This decoding method is optimal on a single time-step level.

To facilite the greedy decoding operation, we define a *GreedySearchDecoder* class. When run, an object of this class takes an input sequence (*input_seq*) of shape *(input_seq length, 1)*, a scalar input length *(input_length)* tensor, and a *max_length* to bound the response sentence length.

The input sentence is inferenced using the following computational graph:

**Computational Graph:**

1. Forward input through encoder model

2. Prepare encoder's final hidden layer to be first hidden input to the decoder

3. Initialize decoder's first input as SOS_TOKEN

4. Initialize tensors to append decoded words to

5. Iteratively decode one word token at a time:

  5.1 Forward pass through decoder
  
  5.2 Obtain most likely word token and its softmax score
  
  5.3 Record token and score
  
  5.4 Prepare current token to be next decoder input
  
6. Return collections of word tokens and scores

> 刘尧：疑问：这又是要干啥子玩意儿？？这么多Decoder啊？另外，decoder到底是指整个Decoder模块，还是Decoder模块中每个time step对应的GRU/LSTM Cell？？？

> 刘尧：只是用于应用环节！即：计算decoder的所有输出以及对应的scores

In [33]:
class GreedySearchDecoder(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, input_seq, input_length, max_length):
        # 1. Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        
        # 2. Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[: decoder.n_layers]
        
        # 3. Initialize decoder input with SOS_TOKEN
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_TOKEN
        
        # 4. Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        
        # 5. Iteratively decode one word token at a time
        for _ in range(max_length):
            # 5.1 Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)  # 疑问：此处的decoder应该是LSTM/GRU Cell吧？
            
            # 5.2 Obtain most likely word token and its softmax value
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            
            # 5.3 Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            
            # 5.4 Prepare current token to be next decoder input
            decoder_input = torch.unsqueeze(decoder_input, 0)
        
        # 6. Return collections of word tokens and scores
        return all_tokens, all_scores

### Inference my text

Having decoding method defined, we can write function ***inference*** for inferencing a string input sentence. *inference* manages the *low-level process* of handling the input sentence.

We first format the sentence as an input batch of word indexes with batch_size=1 by converting the words of the sentence to their corresponding indexes, and transposing the dimensions to prepare the tensor for our models. We also create a *lengths* tensor which contains the length of our input sentence. In this case, *length* is scalar because we are only evaluating one sentence at a time (batch_size==1).

Next, we obtain the decoded response sentence tensor using our *GreedySearchDecoder* object (*searcher*).

Finally, we convert the response's indexes to words and return the list of decoded words.

> **inference**: words of one sentence --> their corresponding indexes --transpose-->  tensor for models  --GreedySearchDecoder--> decoded response indexes --> list of decoded words

***inferenceInput*** acts as the user interface for our chatbot. When called, an input sentence is **normalized in the same way as training data**, and is ultimately fed to the **inference** function to obtain a decoded output sentence. We loop this process, so we can keep chatting until we enter either 'q' or 'quit'.

Finally, if a sentence contains a OOV word, we handle this gracefully by printing an error message and prompting the user.

In [34]:
def inference(encoder, decoder, searcher, vocabulary, sentence, max_length=MAX_LENGTH):
    # words --> indexes
    indexes_batch = [sentenceToIndexes(vocabulary, sentence)]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    lengths = lengths.to(device)
    
    # Transpose dimensions of batch for models
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    input_batch = input_batch.to(device)
    
    # Decode sentence with searcher
    tokens, _ = searcher(input_batch, lengths, max_length)
    
    # indexes --> words
    decoded_words = [vocabulary.index2word[token.item()] for token in tokens]
    
    return decoded_words

In [35]:
def inferenceInput(encoder, decoder, searcher, vocabulary):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('>')
            if input_sentence in ('q', 'quit'):
                break
                
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            
            # Inference sentence
            output_words = inference(encoder, decoder, searcher, vocabulary, input_sentence)
            
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x in ('EOS', 'PAD'))]
            print('Bot:', ' '.join(output_words))
            
        except KeyError:
            print('Error: Encountered unkonw word.')

## 7. Run Model

Regardless of whether we want to train or test the chatbot model, we must **initialize the individual encoder and decoder models**. 

In the following block, we set our desired configurations, choose to start from scratch or set a checkpoint to load from, and build and initialize the models.

In [36]:
# Configure models
model_name = 'cb_model'
attn_method = 'dot'
#attn_method = 'general'
#attn_method = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
checkpoint_file = None
checkpoint_iter = 4000
#checkpoint_file = os.path.join(save_dir, model_name, corpus_name, f'{encoder_n_layers}-{decoder_n_layers}_{hidden_size}', f'{checkpoint_iter}_checkpoint.tar')

In [37]:
# Load model if a checkpoint_file is provided
if checkpoint_file:
    checkpoint = torch.load(checkpoint_file)
    #checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))  # If loading a model trained on GPU to CPU
    encoder_sd = checkpoint['encoder']
    decoder_sd = checkpoint['decoder']
    encoder_optimizer_sd = checkpoint['encoder_optimizer']
    decoder_optimizer_sd = checkpoint['decoder_optimizer']
    embedding_sd = checkpoint['embedding']
    vocabulary.__dict__ = checkpoint['voc_dict']

> Embedding, Encoder, Decoder三者分别独立定义，使用时再结合在一起。训练和应用时一起进行，又可独立拿出来。

In [38]:
print('Building encoder and decoder ...')

# Initialize word embeddings
embedding = nn.Embedding(vocabulary.num_words, hidden_size)
if checkpoint_file:
    embedding.load_state_dict(embedding_sd)
    
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_method, embedding, hidden_size, vocabulary.num_words, decoder_n_layers, dropout)
if checkpoint_file:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
    
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


### Run Training

In [39]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

LuongAttnDecoderRNN(
  (embedding): Embedding(7826, 500)
  (embedding_dropout): Dropout(p=0.1)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
  (attn): Attn()
  (concat): Linear(in_features=1000, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=7826, bias=True)
)

In [40]:
# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if checkpoint_file:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

Building optimizers ...


In [41]:
# Run training iterations
print("Starting Training!")
trainIters(model_name, vocabulary, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, 
           save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, checkpoint_file)

Starting Training!
Initializing ...
Training ...
Iteration: 1; Percent complete:  0.0%; Average loss:  8.9690


KeyboardInterrupt: 

### Run Inference

To chat with your model, run the following block.

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [ ]:
# Begin chatting
inferenceInput(encoder, decoder, searcher, vocabulary)